<a href="https://colab.research.google.com/github/vannguyenhoai43-hash/ecommerce-category-growth-analysis/blob/main/notebooks/02_product_keyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **PHẦN 2: CHUẨN HÓA VÀ GOM NHÓM SẢN PHẨM**

**1. Dữ liệu :** ở cấp độ sản phẩm có quy mô rất lớn (hàng triệu dòng), trong đó mỗi dòng tương ứng với một tên sản phẩm khác nhau.

   Việc phân tích trực tiếp trên từng tên sản phẩm riêng lẻ sẽ:
- Gây nhiễu do trùng lặp về bản chất nhưng khác nhau về cách đặt tên
- Làm phân tích bị phân mảnh, khó nhìn ra xu hướng chính
- Tăng độ phức tạp khi tổng hợp số liệu

**2. Mục tiêu:**
   Chuẩn hóa và gom nhóm sản phẩm dựa trên keyword cốt lõi trong tên sản phẩm, thay vì xử lý từng tên riêng lẻ.

   Ví dụ:

   “nón bảo hiểm xinh đẹp”

   “nón bảo hiểm siêu bền”

   Hai sản phẩm này khác nhau về mô tả chi tiết, nhưng đều thuộc cùng một nhóm bản chất là: “nón bảo hiểm”.

**3. Cách tiếp cận**

- Loại bỏ các từ gây nhiễu trong tên sản phẩm

- Lấy ra 5 từ khóa trong tên

  => Từ đó tổng hợp ADO, GMV hoặc các chỉ số khác theo nhóm thay vì theo từng SKU riêng lẻ

**4. Ý nghĩa phân tích**

- Giảm độ phân mảnh của dữ liệu sản phẩm
- Phản ánh đúng bản chất hành vi tiêu dùng theo nhóm sản phẩm
- Làm rõ động lực tăng trưởng ở cấp độ “nhóm sản phẩm” thay vì từng sản phẩm nhỏ lẻ
- Giúp phân tích trên dữ liệu lớn trở nên hệ thống và có tính tổng quát hơn.


##Loại bỏ các từ gây nhiễu

#### Từ đơn

In [ ]:
noise_words = {
    # Đơn vị đo lường & thông số kỹ thuật
    'v', 'w', 'mah', 'ah', 'mm', 'cm', 'm', 'kg', 'g',
    'lít', 'ml', 'inch', 'size',
    '12v', '24v', '220v', '60w', '100w', '120w',

    # Hình thức đóng gói & số lượng
    'cái', 'chiếc', 'viên', 'hộp', 'thùng', 'gói',
    'cặp', 'đôi', 'set', 'lố', 'bịch', 'cuộn',
    'mét', 'tấm', 'miếng', 'sợi',
    'que', 'cây', 'chai', 'lọ', 'tuýp', 'bình','sỉ','lẻ','viền',
    # Vị trí & hướng
    'trước', 'sau', 'trái', 'phải',
    'trên', 'dưới', 'trong', 'ngoài',
    'giữa', 'bên', 'ngang', 'dọc',
    'đứng', 'nằm', 'gầm', 'hậu',

    # Hành động / mục đích
    'dành', 'cho', 'của', 'lắp', 'độ', 'chế',
    'thay', 'thế', 'sửa', 'chữa',
    'làm', 'tẩy', 'rửa', 'xịt', 'bôi',
    'trơn', 'đánh', 'bóng', 'dán', 'ốp',
    'che', 'đựng', 'treo', 'móc', 'giữ', 'nối', 'nhận','cách','ghi','miễn',

    # Tính chất chung
    'chuyên', 'dụng', 'mini', 'nhỏ', 'lớn', 'to',
    'zin', 'xịn', 'nhập', 'khẩu', 'nội', 'địa',
    'loại', 'tốt', 'dày', 'mọi','qsmotor',

    # Màu sắc
    'đen', 'trắng', 'đỏ', 'xanh', 'vàng',
    'bạc', 'tím', 'hồng', 'cam',
    'nâu', 'xám', 'màu', 'trong', 'suốt',

    # Marketing
    'sale', 'off', 'hot', 'new',
    'mới', 'giảm', 'giá', 'rẻ',
    'siêu', 'free', 'freeship',
    'tặng', 'kèm', 'top', 'best', 'trend','sốc',

    # Khác
    'phụ', 'kiện', 'linh', 'đồ', 'nghề', 'cụ',
    'thiết', 'bị', 'hơi', 'nước', 'khí', 'nén',
    'nhà', 'cửa', 'gia', 'đình',
    'full', 'nhiều', 'mẫu', 'đủ'
}

### Từ đôi

In [ ]:
noise_phrases = [
    # Marketing / bán hàng
    'sale off','bán chạy','thanh lý','chính hãng',
    'giá rẻ','cao cấp','hỏa tốc','bảo hành','giá sốc',

    # Thuộc tính / mô tả
    'đa năng','kim loại','chi tiết',
    'khổ rộng','dạng sóng','lấy ánh sáng',
    'hai đầu','dễ thương','đáng yêu',
    'ánh sáng','tự động','bảo hành',

    # Trạng thái / phạm vi
    'ngẫu nhiên','tất cả',
    'bao gồm','hàng loại 1',

    # Thương hiệu / sàn / địa danh
    'shopee','shopee mall','made in',
    'sài gòn','hà nội',
    'ib shop','gù salaya'

    # Năm
    '2017', '2018', '2019',
    '2020', '2021', '2022',
    '2023', '2024', '2025'
]

In [ ]:
import re
import pandas as pd
import unicodedata

def clean_and_shorten(name, num_keywords=5):
    if pd.isna(name):
        return ''
    # 1. Loại bỏ keycap emoji
    name = re.sub(r'[\d]\ufe0f?\u20e3', ' ', name)

    # 2. Loại bỏ emoji khác
    name = re.sub(r'[\U0001F300-\U0001FAFF]', ' ', name)

    # 3. Chuẩn hoá unicode để tránh lỗi ký tự, vẫn giữ nguyên tiếng Việt có dấu.
    name = unicodedata.normalize('NFKC', name)

    # 4. Viết thường
    name = name.lower()

    # 5. Loại bỏ nội dung trong ngoặc
    name = re.sub(r'[\(\[\{].*?[\)\]\}]', ' ', name)

    # 6. Loại bỏ từ nhiễu từ đôi
    for phrase in noise_phrases:
        name = re.sub(rf'\b{re.escape(phrase)}\b', ' ', name)

    # 7. Thay thế "_" bằng space
    name = name.replace('_', ' ')

    # 8. Loại bỏ emoji & ký tự đặc biệt (GIỮ tiếng Việt)
    name = re.sub(r'[^\w\s]', ' ', name)

    # 9. Tách và bỏ các từ trong từ gây nhiễu : từ đơn
    words = name.split()
    keywords = [w for w in words if w not in noise_words]

    # 10. Bỏ trùng lặp – giữ thứ tự
    seen = set()
    unique_keywords = []
    for w in keywords:
        if w not in seen:
            seen.add(w)
            unique_keywords.append(w)

    return ' '.join(unique_keywords[:num_keywords])

In [ ]:
#Test:
a = '🏆 𝑺𝑨𝑳𝑬 𝑶𝑭𝑭 7️⃣5️⃣% 🔥 (Bán sỉ) Mũ bảo hiểm lưỡi trai Vesp cao cấp, cá tính, dùng cho nam nữ'
clean_and_shorten(a)

'mũ bảo hiểm lưỡi trai'

In [ ]:
#Test:
b = '[HCM], hỏa tốc Thắt Lưng Nam🎁Giá sốc🎁màu ghi, có khóa tự động cao cấp, dây nịt nam mặt xoay 360 độ thể thao'
clean_and_shorten(b)

'thắt lưng nam có khóa'